In [9]:
import random
import argparse
import numpy as np

import torch
import torch.backends.cudnn as cudnn

In [10]:
parser = argparse.ArgumentParser()
# General Settings
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--device', type=str, default='0')

args = parser.parse_args(args=[])

In [11]:
device = 'cuda:' + args.device
args.device = torch.device(device)
torch.cuda.set_device(args.device)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
cudnn.deterministic = True
cudnn.benchmark = False